In [1]:
## 모듈로딩
import torch
import torch.nn as nn 
import torchvision.models as models
from torchinfo import summary

- 사전학습된 모델 로딩 및 설정
    - 모델에 사용할 가중치 및 절편 설정
    - 층별 가중치 및 절편 비활성화 설정

- The inference transforms are available at ResNet50_Weights.IMAGENET1K_V2.transforms and perform the following preprocessing operations: Accepts PIL.Image, batched (B, C, H, W) and single (C, H, W) image torch.Tensor objects. The images are resized to resize_size=[232] using interpolation=InterpolationMode.BILINEAR, followed by a central crop of crop_size=[224]. Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].

In [2]:
## 사전학습된 모델 로딩
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\kdp/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:07<00:00, 13.8MB/s]


In [3]:
# 모델 구조 확인
print(model)
summary(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Layer (type:depth-idx)                   Param #
ResNet                                   --
├─Conv2d: 1-1                            9,408
├─BatchNorm2d: 1-2                       128
├─ReLU: 1-3                              --
├─MaxPool2d: 1-4                         --
├─Sequential: 1-5                        --
│    └─Bottleneck: 2-1                   --
│    │    └─Conv2d: 3-1                  4,096
│    │    └─BatchNorm2d: 3-2             128
│    │    └─Conv2d: 3-3                  36,864
│    │    └─BatchNorm2d: 3-4             128
│    │    └─Conv2d: 3-5                  16,384
│    │    └─BatchNorm2d: 3-6             512
│    │    └─ReLU: 3-7                    --
│    │    └─Sequential: 3-8              16,896
│    └─Bottleneck: 2-2                   --
│    │    └─Conv2d: 3-9                  16,384
│    │    └─BatchNorm2d: 3-10            128
│    │    └─Conv2d: 3-11                 36,864
│    │    └─BatchNorm2d: 3-12            128
│    │    └─Conv2d: 3-13               

In [4]:
for named, param in model.named_parameters():
    print(f'[{named}] {param.shape}')

    # 역전파 시에 업데이트 되지 않도록 설정
    param.requires_grad = False

[conv1.weight] torch.Size([64, 3, 7, 7])
[bn1.weight] torch.Size([64])
[bn1.bias] torch.Size([64])
[layer1.0.conv1.weight] torch.Size([64, 64, 1, 1])
[layer1.0.bn1.weight] torch.Size([64])
[layer1.0.bn1.bias] torch.Size([64])
[layer1.0.conv2.weight] torch.Size([64, 64, 3, 3])
[layer1.0.bn2.weight] torch.Size([64])
[layer1.0.bn2.bias] torch.Size([64])
[layer1.0.conv3.weight] torch.Size([256, 64, 1, 1])
[layer1.0.bn3.weight] torch.Size([256])
[layer1.0.bn3.bias] torch.Size([256])
[layer1.0.downsample.0.weight] torch.Size([256, 64, 1, 1])
[layer1.0.downsample.1.weight] torch.Size([256])
[layer1.0.downsample.1.bias] torch.Size([256])
[layer1.1.conv1.weight] torch.Size([64, 256, 1, 1])
[layer1.1.bn1.weight] torch.Size([64])
[layer1.1.bn1.bias] torch.Size([64])
[layer1.1.conv2.weight] torch.Size([64, 64, 3, 3])
[layer1.1.bn2.weight] torch.Size([64])
[layer1.1.bn2.bias] torch.Size([64])
[layer1.1.conv3.weight] torch.Size([256, 64, 1, 1])
[layer1.1.bn3.weight] torch.Size([256])
[layer1.1.bn3.b

- 사전학습 모델의 분류 관련층을 커스텀화 진행 <hr>

In [6]:
model.layer1

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, ke